Connected to Python 3.10.11

In [1]:
# 1. buy ==1,
# 2. sell ==0

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go

In [3]:
nifty = yf.download('^NSEI')['Adj Close']

## calculated drawdown
cum_max = nifty.cummax()
nifty = nifty.to_frame(name='price')  #Converted to dataframe first
nifty['cum_max'] = cum_max

drawdown = (nifty['price'] - nifty['cum_max'])/nifty['cum_max']*100
nifty['drawdown'] = drawdown
nifty = round(nifty,2)

[*********************100%***********************]  1 of 1 completed


In [4]:
trace = go.Scatter(
    x = nifty.index,
    y = nifty['drawdown'],
    mode = 'lines',
    name = 'Line'
)
layout = go.Layout(
    title = 'Nifty price',
    xaxis = dict(title = 'Date'),
    yaxis = dict(title = 'Price')
)
# fig object by passing both
fig = go.Figure(data=[trace], layout= layout)
fig.show()

In [5]:
sell_drawdown = 5  #5%
buy_bounce= 5      #5%

In [6]:
nifty['cum_min'] =  nifty['price'].cummin()

bounce_up = (nifty['price'] - nifty['cum_min'])/nifty['cum_min']*100
nifty['bounce_up'] = bounce_up
nifty = round(nifty,2)
nifty

## calculating entry and exit price

nifty['entry'] = np.where(nifty['bounce_up'] > buy_bounce, 1,0)  # buy = 1
nifty['exit']   = np.where(nifty['drawdown'] < - (sell_drawdown), 1,0)   # sell  =1

nifty

## position

nifty['position'] = 1
for i in range(len(nifty)):
    nifty['position'] = np.where(nifty['position'].shift() ==1,
                        np.where(nifty['exit'] ==1, 0, nifty['position'].shift()),
                        np.where(nifty['entry'] ==1,1,nifty['position'].shift() ))

nifty = nifty.iloc[2:].copy()
nifty['position'] = nifty['position'].astype(int)

nifty

## entry price

nifty['entry_price'] = nifty['price']

nifty['entry_price'] = nifty['price']

for i in range(1,len(nifty)): # skipping first
    if nifty['position'].iloc[i] == 1:
        if nifty['position'].shift().iloc[i] == 0:
            nifty['entry_price'].iloc[i] = nifty['price'].iloc[i]
        else:
            nifty['entry_price'].iloc[i] = nifty['entry_price'].shift().iloc[i]
    else:
        nifty['entry_price'].iloc[i] = 0

<ipython-input-6-b5eaaedd712f>:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-b5eaaedd712f>:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-b5eaaedd712f>:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
nifty['exit_price'] = 0
for i in range(len(nifty)):
    if (nifty['position'].shift().iloc[i] ==1) and (nifty['position'].iloc[i] ==0) :
        nifty['exit_price'].iloc[i] = nifty['price'].iloc[i]

    else:
        nifty['exit_price'].iloc[i] = 0

## PnL and Pnl_pct
nifty['PnL'] = np.where(nifty['exit_price'] !=0, nifty['exit_price']-nifty['entry_price'].shift(),0)
nifty['PnL_pct'] = (nifty['PnL']/nifty['entry_price'].shift())*100

df2 = nifty[['entry_price', 'exit_price', 'PnL', 'PnL_pct']]
df2
df2 = df2.reset_index()

<ipython-input-7-3ee81e3b90ea>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-3ee81e3b90ea>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
df2['rough'] = np.where(df2['entry_price'] == df2['entry_price'].shift(),0,1)
df2['Date'] = df2['Date'].astype(str)

df2['Date2'] = np.where(
    ((df2['rough'] == 1) & (df2['entry_price'] != 0)), df2['Date'],
    np.where((df2['rough'] == 1) & (df2['exit_price'] != 0), df2['Date'], 0)
)

In [9]:
df2['Date2'] = df2['Date2'].replace(0,None)
df2['Date2'] = df2['Date2'].fillna(method='ffill')
df2.drop('rough', axis=1,inplace=True)
df2

,Date,entry_price,exit_price,PnL,PnL_pct,Date2
0,2007-09-19,4732.35,0.0,0.0,NaN,2007-09-19
1,2007-09-20,4732.35,0.0,0.0,0.0,2007-09-19
2,2007-09-21,4732.35,0.0,0.0,0.0,2007-09-19
3,2007-09-24,4732.35,0.0,0.0,0.0,2007-09-19
4,2007-09-25,4732.35,0.0,0.0,0.0,2007-09-19
...,...,...,...,...,...,...
3867,2023-07-03,17915.05,0.0,0.0,0.0,2023-04-27
3868,2023-07-04,17915.05,0.0,0.0,0.0,2023-04-27
3869,2023-07-05,17915.05,0.0,0.0,0.0,2023-04-27
3870,2023-07-06,17915.05,0.0,0.0,0.0,2023-04-27


In [10]:
df2.head(50)

,Date,entry_price,exit_price,PnL,PnL_pct,Date2
0,2007-09-19,4732.35,0.00,0.00,NaN,2007-09-19
1,2007-09-20,4732.35,0.00,0.00,0.000000,2007-09-19
2,2007-09-21,4732.35,0.00,0.00,0.000000,2007-09-19
3,2007-09-24,4732.35,0.00,0.00,0.000000,2007-09-19
4,2007-09-25,4732.35,0.00,0.00,0.000000,2007-09-19
5,2007-09-26,4732.35,0.00,0.00,0.000000,2007-09-19
6,2007-09-27,4732.35,0.00,0.00,0.000000,2007-09-19
7,2007-09-28,4732.35,0.00,0.00,0.000000,2007-09-19
8,2007-10-01,4732.35,0.00,0.00,0.000000,2007-09-19
9,2007-10-03,4732.35,0.00,0.00,0.000000,2007-09-19


In [11]:
df2['Date2']

0       2007-09-19
1       2007-09-19
2       2007-09-19
3       2007-09-19
4       2007-09-19
           ...    
3867    2023-04-27
3868    2023-04-27
3869    2023-04-27
3870    2023-04-27
3871    2023-04-27
Name: Date2, Length: 3872, dtype: object

In [12]:
df2['Date2'].shift()

0             None
1       2007-09-19
2       2007-09-19
3       2007-09-19
4       2007-09-19
           ...    
3867    2023-04-27
3868    2023-04-27
3869    2023-04-27
3870    2023-04-27
3871    2023-04-27
Name: Date2, Length: 3872, dtype: object

In [13]:
df2['Date2'] = df2['Date2'].shift()

In [14]:
df2

,Date,entry_price,exit_price,PnL,PnL_pct,Date2
0,2007-09-19,4732.35,0.0,0.0,NaN,None
1,2007-09-20,4732.35,0.0,0.0,0.0,2007-09-19
2,2007-09-21,4732.35,0.0,0.0,0.0,2007-09-19
3,2007-09-24,4732.35,0.0,0.0,0.0,2007-09-19
4,2007-09-25,4732.35,0.0,0.0,0.0,2007-09-19
...,...,...,...,...,...,...
3867,2023-07-03,17915.05,0.0,0.0,0.0,2023-04-27
3868,2023-07-04,17915.05,0.0,0.0,0.0,2023-04-27
3869,2023-07-05,17915.05,0.0,0.0,0.0,2023-04-27
3870,2023-07-06,17915.05,0.0,0.0,0.0,2023-04-27


In [15]:
df2.head()

,Date,entry_price,exit_price,PnL,PnL_pct,Date2
0,2007-09-19,4732.35,0.0,0.0,NaN,None
1,2007-09-20,4732.35,0.0,0.0,0.0,2007-09-19
2,2007-09-21,4732.35,0.0,0.0,0.0,2007-09-19
3,2007-09-24,4732.35,0.0,0.0,0.0,2007-09-19
4,2007-09-25,4732.35,0.0,0.0,0.0,2007-09-19


In [16]:
df2.head(50)

,Date,entry_price,exit_price,PnL,PnL_pct,Date2
0,2007-09-19,4732.35,0.00,0.00,NaN,None
1,2007-09-20,4732.35,0.00,0.00,0.000000,2007-09-19
2,2007-09-21,4732.35,0.00,0.00,0.000000,2007-09-19
3,2007-09-24,4732.35,0.00,0.00,0.000000,2007-09-19
4,2007-09-25,4732.35,0.00,0.00,0.000000,2007-09-19
5,2007-09-26,4732.35,0.00,0.00,0.000000,2007-09-19
6,2007-09-27,4732.35,0.00,0.00,0.000000,2007-09-19
7,2007-09-28,4732.35,0.00,0.00,0.000000,2007-09-19
8,2007-10-01,4732.35,0.00,0.00,0.000000,2007-09-19
9,2007-10-03,4732.35,0.00,0.00,0.000000,2007-09-19


In [17]:
df2['entry_price_shifted'] = df2['entry_price'].shift()

In [18]:
df2['Date2'] = df2['Date2'].shift()
df2['entry_price_shifted'] = df2['entry_price'].shift()

In [19]:
df2

,Date,entry_price,exit_price,PnL,PnL_pct,Date2,entry_price_shifted
0,2007-09-19,4732.35,0.0,0.0,NaN,None,NaN
1,2007-09-20,4732.35,0.0,0.0,0.0,None,4732.35
2,2007-09-21,4732.35,0.0,0.0,0.0,2007-09-19,4732.35
3,2007-09-24,4732.35,0.0,0.0,0.0,2007-09-19,4732.35
4,2007-09-25,4732.35,0.0,0.0,0.0,2007-09-19,4732.35
...,...,...,...,...,...,...,...
3867,2023-07-03,17915.05,0.0,0.0,0.0,2023-04-27,17915.05
3868,2023-07-04,17915.05,0.0,0.0,0.0,2023-04-27,17915.05
3869,2023-07-05,17915.05,0.0,0.0,0.0,2023-04-27,17915.05
3870,2023-07-06,17915.05,0.0,0.0,0.0,2023-04-27,17915.05


In [20]:
df2.head(50)

,Date,entry_price,exit_price,PnL,PnL_pct,Date2,entry_price_shifted
0,2007-09-19,4732.35,0.00,0.00,NaN,None,NaN
1,2007-09-20,4732.35,0.00,0.00,0.000000,None,4732.35
2,2007-09-21,4732.35,0.00,0.00,0.000000,2007-09-19,4732.35
3,2007-09-24,4732.35,0.00,0.00,0.000000,2007-09-19,4732.35
4,2007-09-25,4732.35,0.00,0.00,0.000000,2007-09-19,4732.35
5,2007-09-26,4732.35,0.00,0.00,0.000000,2007-09-19,4732.35
6,2007-09-27,4732.35,0.00,0.00,0.000000,2007-09-19,4732.35
7,2007-09-28,4732.35,0.00,0.00,0.000000,2007-09-19,4732.35
8,2007-10-01,4732.35,0.00,0.00,0.000000,2007-09-19,4732.35
9,2007-10-03,4732.35,0.00,0.00,0.000000,2007-09-19,4732.35


In [21]:
df2.reanme(columns={'Date2': 'entry_date'})

AttributeError: 'DataFrame' object has no attribute 'reanme'

In [22]:
df2.rename(columns={'Date2': 'entry_date'})

,Date,entry_price,exit_price,PnL,PnL_pct,entry_date,entry_price_shifted
0,2007-09-19,4732.35,0.0,0.0,NaN,None,NaN
1,2007-09-20,4732.35,0.0,0.0,0.0,None,4732.35
2,2007-09-21,4732.35,0.0,0.0,0.0,2007-09-19,4732.35
3,2007-09-24,4732.35,0.0,0.0,0.0,2007-09-19,4732.35
4,2007-09-25,4732.35,0.0,0.0,0.0,2007-09-19,4732.35
...,...,...,...,...,...,...,...
3867,2023-07-03,17915.05,0.0,0.0,0.0,2023-04-27,17915.05
3868,2023-07-04,17915.05,0.0,0.0,0.0,2023-04-27,17915.05
3869,2023-07-05,17915.05,0.0,0.0,0.0,2023-04-27,17915.05
3870,2023-07-06,17915.05,0.0,0.0,0.0,2023-04-27,17915.05


In [23]:
df2.rename(columns={'Date2': 'entry_date'}, inplace=True)

In [24]:
df2

,Date,entry_price,exit_price,PnL,PnL_pct,entry_date,entry_price_shifted
0,2007-09-19,4732.35,0.0,0.0,NaN,None,NaN
1,2007-09-20,4732.35,0.0,0.0,0.0,None,4732.35
2,2007-09-21,4732.35,0.0,0.0,0.0,2007-09-19,4732.35
3,2007-09-24,4732.35,0.0,0.0,0.0,2007-09-19,4732.35
4,2007-09-25,4732.35,0.0,0.0,0.0,2007-09-19,4732.35
...,...,...,...,...,...,...,...
3867,2023-07-03,17915.05,0.0,0.0,0.0,2023-04-27,17915.05
3868,2023-07-04,17915.05,0.0,0.0,0.0,2023-04-27,17915.05
3869,2023-07-05,17915.05,0.0,0.0,0.0,2023-04-27,17915.05
3870,2023-07-06,17915.05,0.0,0.0,0.0,2023-04-27,17915.05


In [25]:
df2.rename(columns={'Date': 'exit_date'}, inplace=True)

In [26]:
df2

,exit_date,entry_price,exit_price,PnL,PnL_pct,entry_date,entry_price_shifted
0,2007-09-19,4732.35,0.0,0.0,NaN,None,NaN
1,2007-09-20,4732.35,0.0,0.0,0.0,None,4732.35
2,2007-09-21,4732.35,0.0,0.0,0.0,2007-09-19,4732.35
3,2007-09-24,4732.35,0.0,0.0,0.0,2007-09-19,4732.35
4,2007-09-25,4732.35,0.0,0.0,0.0,2007-09-19,4732.35
...,...,...,...,...,...,...,...
3867,2023-07-03,17915.05,0.0,0.0,0.0,2023-04-27,17915.05
3868,2023-07-04,17915.05,0.0,0.0,0.0,2023-04-27,17915.05
3869,2023-07-05,17915.05,0.0,0.0,0.0,2023-04-27,17915.05
3870,2023-07-06,17915.05,0.0,0.0,0.0,2023-04-27,17915.05


In [27]:
df2[['entry_date', 'exit_date', 'entry_price_shifted','PnL', 'PnL_pct']]

,entry_date,exit_date,entry_price_shifted,PnL,PnL_pct
0,None,2007-09-19,NaN,0.0,NaN
1,None,2007-09-20,4732.35,0.0,0.0
2,2007-09-19,2007-09-21,4732.35,0.0,0.0
3,2007-09-19,2007-09-24,4732.35,0.0,0.0
4,2007-09-19,2007-09-25,4732.35,0.0,0.0
...,...,...,...,...,...
3867,2023-04-27,2023-07-03,17915.05,0.0,0.0
3868,2023-04-27,2023-07-04,17915.05,0.0,0.0
3869,2023-04-27,2023-07-05,17915.05,0.0,0.0
3870,2023-04-27,2023-07-06,17915.05,0.0,0.0


In [28]:
df3 = df2[['entry_date', 'exit_date', 'entry_price_shifted','PnL', 'PnL_pct']]

In [29]:
df3

,entry_date,exit_date,entry_price_shifted,PnL,PnL_pct
0,None,2007-09-19,NaN,0.0,NaN
1,None,2007-09-20,4732.35,0.0,0.0
2,2007-09-19,2007-09-21,4732.35,0.0,0.0
3,2007-09-19,2007-09-24,4732.35,0.0,0.0
4,2007-09-19,2007-09-25,4732.35,0.0,0.0
...,...,...,...,...,...
3867,2023-04-27,2023-07-03,17915.05,0.0,0.0
3868,2023-04-27,2023-07-04,17915.05,0.0,0.0
3869,2023-04-27,2023-07-05,17915.05,0.0,0.0
3870,2023-04-27,2023-07-06,17915.05,0.0,0.0


In [30]:
df2['PnL']

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
3867    0.0
3868    0.0
3869    0.0
3870    0.0
3871    0.0
Name: PnL, Length: 3872, dtype: float64

In [31]:
df2['PnL'].isna()

0       False
1       False
2       False
3       False
4       False
        ...  
3867    False
3868    False
3869    False
3870    False
3871    False
Name: PnL, Length: 3872, dtype: bool

In [32]:
df2['PnL'].isna().sum()

0

In [34]:
df2['PnL'] !=0

0       False
1       False
2       False
3       False
4       False
        ...  
3867    False
3868    False
3869    False
3870    False
3871    False
Name: PnL, Length: 3872, dtype: bool

In [35]:
df2[df2['PnL'] !=0]

,exit_date,entry_price,exit_price,PnL,PnL_pct,entry_date,entry_price_shifted
20,2007-10-18,0.0,5351.00,618.65,13.072786,2007-09-19,4732.35
22,2007-10-22,0.0,5184.00,-31.30,-0.600157,2007-10-18,5215.30
36,2007-11-12,0.0,5617.10,143.40,2.619800,2007-10-23,5473.70
43,2007-11-21,0.0,5561.05,-134.35,-2.358921,2007-11-13,5695.40
45,2007-11-23,0.0,5608.60,89.25,1.617038,2007-11-21,5519.35
...,...,...,...,...,...,...,...
3813,2023-04-13,0.0,17828.00,15.60,0.087579,2023-04-11,17812.40
3815,2023-04-18,0.0,17660.15,-46.70,-0.263740,2023-04-13,17706.85
3817,2023-04-20,0.0,17624.45,5.70,0.032352,2023-04-18,17618.75
3819,2023-04-24,0.0,17743.40,119.35,0.677200,2023-04-20,17624.05


In [36]:
df3 = df2[['entry_date', 'exit_date', 'entry_price_shifted','PnL', 'PnL_pct']]
df2 = df2[df2['PnL'] !=0]

In [37]:
df3

,entry_date,exit_date,entry_price_shifted,PnL,PnL_pct
0,None,2007-09-19,NaN,0.0,NaN
1,None,2007-09-20,4732.35,0.0,0.0
2,2007-09-19,2007-09-21,4732.35,0.0,0.0
3,2007-09-19,2007-09-24,4732.35,0.0,0.0
4,2007-09-19,2007-09-25,4732.35,0.0,0.0
...,...,...,...,...,...
3867,2023-04-27,2023-07-03,17915.05,0.0,0.0
3868,2023-04-27,2023-07-04,17915.05,0.0,0.0
3869,2023-04-27,2023-07-05,17915.05,0.0,0.0
3870,2023-04-27,2023-07-06,17915.05,0.0,0.0


In [38]:
df3.head(50)

,entry_date,exit_date,entry_price_shifted,PnL,PnL_pct
0,None,2007-09-19,NaN,0.00,NaN
1,None,2007-09-20,4732.35,0.00,0.000000
2,2007-09-19,2007-09-21,4732.35,0.00,0.000000
3,2007-09-19,2007-09-24,4732.35,0.00,0.000000
4,2007-09-19,2007-09-25,4732.35,0.00,0.000000
5,2007-09-19,2007-09-26,4732.35,0.00,0.000000
6,2007-09-19,2007-09-27,4732.35,0.00,0.000000
7,2007-09-19,2007-09-28,4732.35,0.00,0.000000
8,2007-09-19,2007-10-01,4732.35,0.00,0.000000
9,2007-09-19,2007-10-03,4732.35,0.00,0.000000


In [39]:
df3 = df2[['entry_date', 'exit_date', 'entry_price_shifted','PnL', 'PnL_pct']]
df3 = df2[df2['PnL'] !=0]

In [40]:
df3.head(50)

,exit_date,entry_price,exit_price,PnL,PnL_pct,entry_date,entry_price_shifted
20,2007-10-18,0.0,5351.00,618.65,13.072786,2007-09-19,4732.35
22,2007-10-22,0.0,5184.00,-31.30,-0.600157,2007-10-18,5215.30
36,2007-11-12,0.0,5617.10,143.40,2.619800,2007-10-23,5473.70
43,2007-11-21,0.0,5561.05,-134.35,-2.358921,2007-11-13,5695.40
45,2007-11-23,0.0,5608.60,89.25,1.617038,2007-11-21,5519.35
48,2007-11-28,0.0,5617.55,-114.15,-1.991556,2007-11-26,5731.70
61,2007-12-17,0.0,5777.00,142.40,2.527242,2007-11-29,5634.60
63,2007-12-19,0.0,5751.15,8.85,0.154119,2007-12-17,5742.30
81,2008-01-16,0.0,5935.75,169.25,2.935056,2007-12-20,5766.50
83,2008-01-18,0.0,5705.30,-207.90,-3.515863,2008-01-16,5913.20


In [41]:
#%% note:
# 1. buy ==1,
# 2. sell ==0

In [42]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go

In [43]:
nifty = yf.download('^NSEI')['Adj Close']

## calculated drawdown
cum_max = nifty.cummax()
nifty = nifty.to_frame(name='price')  #Converted to dataframe first
nifty['cum_max'] = cum_max

drawdown = (nifty['price'] - nifty['cum_max'])/nifty['cum_max']*100
nifty['drawdown'] = drawdown
nifty = round(nifty,2)

[*********************100%***********************]  1 of 1 completed


In [44]:
trace = go.Scatter(
    x = nifty.index,
    y = nifty['drawdown'],
    mode = 'lines',
    name = 'Line'
)
layout = go.Layout(
    title = 'Nifty price',
    xaxis = dict(title = 'Date'),
    yaxis = dict(title = 'Price')
)
# fig object by passing both
fig = go.Figure(data=[trace], layout= layout)
fig.show()

In [45]:
sell_drawdown = 5  #5%
buy_bounce= 5      #5%

In [46]:
nifty['cum_min'] =  nifty['price'].cummin()

bounce_up = (nifty['price'] - nifty['cum_min'])/nifty['cum_min']*100
nifty['bounce_up'] = bounce_up
nifty = round(nifty,2)
nifty



## calculating entry and exit price
nifty['entry'] = np.where(nifty['bounce_up'] > buy_bounce, 1,0)  # buy = 1
nifty['exit']   = np.where(nifty['drawdown'] < - (sell_drawdown), 1,0)   # sell  =1



## position
nifty['position'] = 1
for i in range(len(nifty)):
    nifty['position'] = np.where(nifty['position'].shift() ==1,
                        np.where(nifty['exit'] ==1, 0, nifty['position'].shift()),
                        np.where(nifty['entry'] ==1,1,nifty['position'].shift() ))

nifty = nifty.iloc[2:].copy()
nifty['position'] = nifty['position'].astype(int)

## entry price
nifty['entry_price'] = nifty['price']
for i in range(1,len(nifty)): # skipping first
    if nifty['position'].iloc[i] == 1:
        if nifty['position'].shift().iloc[i] == 0:
            nifty['entry_price'].iloc[i] = nifty['price'].iloc[i]
        else:
            nifty['entry_price'].iloc[i] = nifty['entry_price'].shift().iloc[i]
    else:
        nifty['entry_price'].iloc[i] = 0

<ipython-input-46-11bcbdeb5967>:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-46-11bcbdeb5967>:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-46-11bcbdeb5967>:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [47]:
nifty['exit_price'] = 0
for i in range(len(nifty)):
    if (nifty['position'].shift().iloc[i] ==1) and (nifty['position'].iloc[i] ==0) :
        nifty['exit_price'].iloc[i] = nifty['price'].iloc[i]

    else:
        nifty['exit_price'].iloc[i] = 0

## PnL and Pnl_pct
nifty['PnL'] = np.where(nifty['exit_price'] !=0, nifty['exit_price']-nifty['entry_price'].shift(),0)
nifty['PnL_pct'] = (nifty['PnL']/nifty['entry_price'].shift())*100

df2 = nifty[['entry_price', 'exit_price', 'PnL', 'PnL_pct']]
df2
df2 = df2.reset_index()

<ipython-input-47-9365fe545f49>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-47-9365fe545f49>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [48]:
df2['rough'] = np.where(df2['entry_price'] == df2['entry_price'].shift(),0,1)
df2['Date'] = df2['Date'].astype(str)

df2['Date2'] = np.where(
    ((df2['rough'] == 1) & (df2['entry_price'] != 0)), df2['Date'],
    np.where((df2['rough'] == 1) & (df2['exit_price'] != 0), df2['Date'], 0)
)

In [49]:
df2['Date2'] = df2['Date2'].replace(0,None)
df2['Date2'] = df2['Date2'].fillna(method='ffill')
df2.drop('rough', axis=1,inplace=True)
df2

,Date,entry_price,exit_price,PnL,PnL_pct,Date2
0,2007-09-19,4732.35,0.0,0.0,NaN,2007-09-19
1,2007-09-20,4732.35,0.0,0.0,0.0,2007-09-19
2,2007-09-21,4732.35,0.0,0.0,0.0,2007-09-19
3,2007-09-24,4732.35,0.0,0.0,0.0,2007-09-19
4,2007-09-25,4732.35,0.0,0.0,0.0,2007-09-19
...,...,...,...,...,...,...
3867,2023-07-03,17915.05,0.0,0.0,0.0,2023-04-27
3868,2023-07-04,17915.05,0.0,0.0,0.0,2023-04-27
3869,2023-07-05,17915.05,0.0,0.0,0.0,2023-04-27
3870,2023-07-06,17915.05,0.0,0.0,0.0,2023-04-27


In [50]:
df2['Date2'] = df2['Date2'].shift()
df2['entry_price_shifted'] = df2['entry_price'].shift()

In [51]:
df2.rename(columns={'Date2': 'entry_date'}, inplace=True)
df2.rename(columns={'Date': 'exit_date'}, inplace=True)

In [52]:
df3 = df2[['entry_date', 'exit_date', 'entry_price_shifted','PnL', 'PnL_pct']]
df3 = df2[df2['PnL'] !=0]

In [53]:
df3

,exit_date,entry_price,exit_price,PnL,PnL_pct,entry_date,entry_price_shifted
20,2007-10-18,0.0,5351.00,618.65,13.072786,2007-09-19,4732.35
22,2007-10-22,0.0,5184.00,-31.30,-0.600157,2007-10-19,5215.30
36,2007-11-12,0.0,5617.10,143.40,2.619800,2007-10-23,5473.70
43,2007-11-21,0.0,5561.05,-134.35,-2.358921,2007-11-13,5695.40
45,2007-11-23,0.0,5608.60,89.25,1.617038,2007-11-22,5519.35
...,...,...,...,...,...,...,...
3813,2023-04-13,0.0,17828.00,15.60,0.087579,2023-04-12,17812.40
3815,2023-04-18,0.0,17660.15,-46.70,-0.263740,2023-04-17,17706.85
3817,2023-04-20,0.0,17624.45,5.70,0.032352,2023-04-19,17618.75
3819,2023-04-24,0.0,17743.40,119.35,0.677200,2023-04-21,17624.05


In [54]:
df2.head(50)

,exit_date,entry_price,exit_price,PnL,PnL_pct,entry_date,entry_price_shifted
0,2007-09-19,4732.35,0.00,0.00,NaN,None,NaN
1,2007-09-20,4732.35,0.00,0.00,0.000000,2007-09-19,4732.35
2,2007-09-21,4732.35,0.00,0.00,0.000000,2007-09-19,4732.35
3,2007-09-24,4732.35,0.00,0.00,0.000000,2007-09-19,4732.35
4,2007-09-25,4732.35,0.00,0.00,0.000000,2007-09-19,4732.35
5,2007-09-26,4732.35,0.00,0.00,0.000000,2007-09-19,4732.35
6,2007-09-27,4732.35,0.00,0.00,0.000000,2007-09-19,4732.35
7,2007-09-28,4732.35,0.00,0.00,0.000000,2007-09-19,4732.35
8,2007-10-01,4732.35,0.00,0.00,0.000000,2007-09-19,4732.35
9,2007-10-03,4732.35,0.00,0.00,0.000000,2007-09-19,4732.35


In [55]:
df3 = df2[['entry_date', 'exit_date', 'entry_price_shifted','PnL', 'PnL_pct']]
df3 = df3[df3['PnL'] !=0]

In [56]:
df3.head(50)

,entry_date,exit_date,entry_price_shifted,PnL,PnL_pct
20,2007-09-19,2007-10-18,4732.35,618.65,13.072786
22,2007-10-19,2007-10-22,5215.30,-31.30,-0.600157
36,2007-10-23,2007-11-12,5473.70,143.40,2.619800
43,2007-11-13,2007-11-21,5695.40,-134.35,-2.358921
45,2007-11-22,2007-11-23,5519.35,89.25,1.617038
48,2007-11-26,2007-11-28,5731.70,-114.15,-1.991556
61,2007-11-29,2007-12-17,5634.60,142.40,2.527242
63,2007-12-18,2007-12-19,5742.30,8.85,0.154119
81,2007-12-20,2008-01-16,5766.50,169.25,2.935056
83,2008-01-17,2008-01-18,5913.20,-207.90,-3.515863


In [57]:
df3.to_excel('file.xlsx')